### 기상청 날씨 데이터 
* beautifulsoup의 find() , find_all() 함수를 사용해서 파싱하기
* 파싱한 데이터를 list와 dict 객체에 저장하기
* json file로 저장하고, 읽기
* list에 저장된 데이터를 pandas의 DataFrame 객체에 저장하기

In [103]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = 'http://www.weather.go.kr/weather/forecast/mid-term-rss3.jsp'

res = requests.get(url)
print(res.status_code)
html = res.text
soup = BeautifulSoup(html,'html.parser')

#find() 함수 사용
title = soup.find('title')
print(type(title), title, title.text)

title_all = soup.find_all('title')
print(type(title_all), title_all)
for item in title_all:
    print(item.string)
    


200
<class 'bs4.element.Tag'> <title>기상청 육상 중기예보</title> 기상청 육상 중기예보
<class 'bs4.element.ResultSet'> [<title>기상청 육상 중기예보</title>, <title>전국 육상 중기예보 - 2020년 10월 21일 (수)요일 06:00 발표</title>, <title>전국 육상중기예보</title>]
기상청 육상 중기예보
전국 육상 중기예보 - 2020년 10월 21일 (수)요일 06:00 발표
전국 육상중기예보


In [104]:
for header in soup.find_all('header'):
    title = header.find('title').text
    tm = header.find('tm').text
    wf = header.find('wf').string
    print(title, tm, wf)
    #print(header)

전국 육상중기예보 202010210600 ○ (기온) 아침 기온(24~25일 제외)은 4~13도로 어제(6~14도)와 비슷하거나 조금 낮겠고, 낮 기온은 14~22도로 어제(20~24도)보다 낮겠습니다. <br />          한편, 24일(토)~25일(일) 아침 기온은 0~11도로 어제보다 낮겠습니다. <br />○ (건조) 대체로 맑은 날씨가 이어지면서 대기가 건조하겠으니, 산불 등 화재 예방에 유의하기 바랍니다.<br />○ (해상) 24일(토) 동해상과 제주도해상은 물결이 2.0~4.0m로 매우 높게 일겠습니다.  <br />          또한, 24일(토) 동해안은 너울에 의한 높은 물결이 방파제나 해안도로를 넘는 곳이 있겠습니다.<br />○ (주말전망) 24일(토)과 25일(일)은 전국이 대체로 맑겠습니다. 아침 기온은 0~11도, 낮 기온은 14~21도의 분포가 되겠습니다.


In [105]:
#location 태그가 41개 
print(len(soup.find_all('location',attrs={"wl_ver":"3"})))
location_one = soup.find('location',attrs={"wl_ver":"3"})
province = location_one.find('province').text
city = location_one.find('city').text
#print(province, city)
print(len(location_one.find_all('data')))
for data in location_one.find_all('data'):
    print('------')
    mode = data.find('mode').text
    tmef = data.find('tmef').text
    wf = data.find('wf').text
    tmn = data.find('tmn').text
    tmx = data.find('tmx').text
    print(mode, tmef, wf, tmn, tmx)
    #print(data)

41
13
------
A02 2020-10-24 00:00 맑음 3 14
------
A02 2020-10-24 12:00 맑음 3 14
------
A02 2020-10-25 00:00 맑음 7 18
------
A02 2020-10-25 12:00 구름많음 7 18
------
A02 2020-10-26 00:00 맑음 9 19
------
A02 2020-10-26 12:00 맑음 9 19
------
A02 2020-10-27 00:00 구름많음 9 18
------
A02 2020-10-27 12:00 맑음 9 18
------
A02 2020-10-28 00:00 맑음 8 17
------
A02 2020-10-28 12:00 맑음 8 17
------
A01 2020-10-29 00:00 맑음 8 17
------
A01 2020-10-30 00:00 맑음 7 18
------
A01 2020-10-31 00:00 맑음 9 19


In [106]:
location_dict = {}

location_one = soup.find('location',attrs={"wl_ver":"3"})
province = location_one.find('province').text
city = location_one.find('city').text

location_dict['province'] = province
location_dict['city'] = city

data_list = []
for data in location_one.find_all('data'):
    data_dict = {}
    data_dict['mode'] = data.find('mode').text
    data_dict['tmef'] = data.find('tmef').text
    data_dict['wf'] = data.find('wf').text
    data_dict['tmn'] = data.find('tmn').text
    data_dict['tmx'] = data.find('tmx').text
    data_list.append(data_dict)

location_dict['datas'] = data_list
print(location_dict)    

{'province': '서울ㆍ인천ㆍ경기도', 'city': '서울', 'datas': [{'mode': 'A02', 'tmef': '2020-10-24 00:00', 'wf': '맑음', 'tmn': '3', 'tmx': '14'}, {'mode': 'A02', 'tmef': '2020-10-24 12:00', 'wf': '맑음', 'tmn': '3', 'tmx': '14'}, {'mode': 'A02', 'tmef': '2020-10-25 00:00', 'wf': '맑음', 'tmn': '7', 'tmx': '18'}, {'mode': 'A02', 'tmef': '2020-10-25 12:00', 'wf': '구름많음', 'tmn': '7', 'tmx': '18'}, {'mode': 'A02', 'tmef': '2020-10-26 00:00', 'wf': '맑음', 'tmn': '9', 'tmx': '19'}, {'mode': 'A02', 'tmef': '2020-10-26 12:00', 'wf': '맑음', 'tmn': '9', 'tmx': '19'}, {'mode': 'A02', 'tmef': '2020-10-27 00:00', 'wf': '구름많음', 'tmn': '9', 'tmx': '18'}, {'mode': 'A02', 'tmef': '2020-10-27 12:00', 'wf': '맑음', 'tmn': '9', 'tmx': '18'}, {'mode': 'A02', 'tmef': '2020-10-28 00:00', 'wf': '맑음', 'tmn': '8', 'tmx': '17'}, {'mode': 'A02', 'tmef': '2020-10-28 12:00', 'wf': '맑음', 'tmn': '8', 'tmx': '17'}, {'mode': 'A01', 'tmef': '2020-10-29 00:00', 'wf': '맑음', 'tmn': '8', 'tmx': '17'}, {'mode': 'A01', 'tmef': '2020-10-30 00:00', 

### 전국의 날씨정보를 dict와 list에 저장하기

In [107]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = 'http://www.weather.go.kr/weather/forecast/mid-term-rss3.jsp'

res = requests.get(url)
print(res.status_code)
html = res.text
soup = BeautifulSoup(html,'html.parser')

#41개의 location 정보를 저장할 list 
location_list = list()
locations = soup.find_all('location', attrs={'wl_ver':'3'})
print(len(locations))
for location in locations:
    location_dict = dict()
    location_dict['province'] = location.find('province').text
    location_dict['city'] = location.find('city').text
    
    datas = location.find_all('data')
    #print(len(datas))
    #13개의 data 정보를 저장할 list
    data_list = list()
    for data in datas:
        #1개의 data 정보를 저장할 dict
        data_dict = dict()
        data_dict['mode'] = data.find('mode').text
        data_dict['tmef'] = data.find('tmef').text
        data_dict['wf'] = data.find('wf').text
        data_dict['tmn'] = data.find('tmn').text
        data_dict['tmx'] = data.find('tmx').text
        data_list.append(data_dict)
    
    location_dict['datas'] = data_list
    
    location_list.append(location_dict)
print(location_list)

200
41
[{'province': '서울ㆍ인천ㆍ경기도', 'city': '서울', 'datas': [{'mode': 'A02', 'tmef': '2020-10-24 00:00', 'wf': '맑음', 'tmn': '3', 'tmx': '14'}, {'mode': 'A02', 'tmef': '2020-10-24 12:00', 'wf': '맑음', 'tmn': '3', 'tmx': '14'}, {'mode': 'A02', 'tmef': '2020-10-25 00:00', 'wf': '맑음', 'tmn': '7', 'tmx': '18'}, {'mode': 'A02', 'tmef': '2020-10-25 12:00', 'wf': '구름많음', 'tmn': '7', 'tmx': '18'}, {'mode': 'A02', 'tmef': '2020-10-26 00:00', 'wf': '맑음', 'tmn': '9', 'tmx': '19'}, {'mode': 'A02', 'tmef': '2020-10-26 12:00', 'wf': '맑음', 'tmn': '9', 'tmx': '19'}, {'mode': 'A02', 'tmef': '2020-10-27 00:00', 'wf': '구름많음', 'tmn': '9', 'tmx': '18'}, {'mode': 'A02', 'tmef': '2020-10-27 12:00', 'wf': '맑음', 'tmn': '9', 'tmx': '18'}, {'mode': 'A02', 'tmef': '2020-10-28 00:00', 'wf': '맑음', 'tmn': '8', 'tmx': '17'}, {'mode': 'A02', 'tmef': '2020-10-28 12:00', 'wf': '맑음', 'tmn': '8', 'tmx': '17'}, {'mode': 'A01', 'tmef': '2020-10-29 00:00', 'wf': '맑음', 'tmn': '8', 'tmx': '17'}, {'mode': 'A01', 'tmef': '2020-10-30 

### Json File로 저장하고 다시 읽기

In [108]:
import json

with open('data/weater_1021.json','w') as file:
    json.dump(location_list, file)

In [109]:
import json

with open('data/weater_1021.json','r') as file:
    contents = file.read()
    weather_json = json.loads(contents)

weather_json    

[{'province': '서울ㆍ인천ㆍ경기도',
  'city': '서울',
  'datas': [{'mode': 'A02',
    'tmef': '2020-10-24 00:00',
    'wf': '맑음',
    'tmn': '3',
    'tmx': '14'},
   {'mode': 'A02',
    'tmef': '2020-10-24 12:00',
    'wf': '맑음',
    'tmn': '3',
    'tmx': '14'},
   {'mode': 'A02',
    'tmef': '2020-10-25 00:00',
    'wf': '맑음',
    'tmn': '7',
    'tmx': '18'},
   {'mode': 'A02',
    'tmef': '2020-10-25 12:00',
    'wf': '구름많음',
    'tmn': '7',
    'tmx': '18'},
   {'mode': 'A02',
    'tmef': '2020-10-26 00:00',
    'wf': '맑음',
    'tmn': '9',
    'tmx': '19'},
   {'mode': 'A02',
    'tmef': '2020-10-26 12:00',
    'wf': '맑음',
    'tmn': '9',
    'tmx': '19'},
   {'mode': 'A02',
    'tmef': '2020-10-27 00:00',
    'wf': '구름많음',
    'tmn': '9',
    'tmx': '18'},
   {'mode': 'A02',
    'tmef': '2020-10-27 12:00',
    'wf': '맑음',
    'tmn': '9',
    'tmx': '18'},
   {'mode': 'A02',
    'tmef': '2020-10-28 00:00',
    'wf': '맑음',
    'tmn': '8',
    'tmx': '17'},
   {'mode': 'A02',
    'tmef': '202

### Pandas를 사용해서 표데이터 만들기
* DataFrame 객체생성
* Series 객체

In [110]:
import pandas as pd

weather_df = pd.read_json('data/weater_1021.json')
print(weather_df.shape)
weather_df.head()

(41, 3)


,province,city,datas
0,서울ㆍ인천ㆍ경기도,서울,"[{'mode': 'A02', 'tmef': '2020-10-24 00:00', '..."
1,서울ㆍ인천ㆍ경기도,인천,"[{'mode': 'A02', 'tmef': '2020-10-24 00:00', '..."
2,서울ㆍ인천ㆍ경기도,수원,"[{'mode': 'A02', 'tmef': '2020-10-24 00:00', '..."
3,서울ㆍ인천ㆍ경기도,파주,"[{'mode': 'A02', 'tmef': '2020-10-24 00:00', '..."
4,서울ㆍ인천ㆍ경기도,이천,"[{'mode': 'A02', 'tmef': '2020-10-24 00:00', '..."


In [111]:
weather_df.tail()

,province,city,datas
36,대구ㆍ경상북도,경주,"[{'mode': 'A02', 'tmef': '2020-10-24 00:00', '..."
37,대구ㆍ경상북도,울진,"[{'mode': 'A02', 'tmef': '2020-10-24 00:00', '..."
38,대구ㆍ경상북도,울릉도,"[{'mode': 'A02', 'tmef': '2020-10-24 00:00', '..."
39,제주도,제주,"[{'mode': 'A02', 'tmef': '2020-10-24 00:00', '..."
40,제주도,서귀포,"[{'mode': 'A02', 'tmef': '2020-10-24 00:00', '..."


In [112]:
weather_df['province'].unique()

array(['서울ㆍ인천ㆍ경기도', '강원도영서', '강원도영동', '대전ㆍ세종ㆍ충청남도', '충청북도', '광주ㆍ전라남도',
       '전라북도', '부산ㆍ울산ㆍ경상남도', '대구ㆍ경상북도', '제주도'], dtype=object)

In [113]:
weather_df['city'].unique()

array(['서울', '인천', '수원', '파주', '이천', '평택', '춘천', '원주', '강릉', '대전', '세종',
       '홍성', '청주', '충주', '영동', '광주', '목포', '여수', '순천', '광양', '나주', '전주',
       '군산', '정읍', '남원', '고창', '무주', '부산', '울산', '창원', '진주', '거창', '통영',
       '대구', '안동', '포항', '경주', '울진', '울릉도', '제주', '서귀포'], dtype=object)

In [114]:
weather_df['province'].value_counts()

대구ㆍ경상북도       6
광주ㆍ전라남도       6
서울ㆍ인천ㆍ경기도     6
전라북도          6
부산ㆍ울산ㆍ경상남도    6
대전ㆍ세종ㆍ충청남도    3
충청북도          3
제주도           2
강원도영서         2
강원도영동         1
Name: province, dtype: int64

In [115]:
#서울
weather_df.loc[weather_df['city'] == '서울']

,province,city,datas
0,서울ㆍ인천ㆍ경기도,서울,"[{'mode': 'A02', 'tmef': '2020-10-24 00:00', '..."


In [116]:
#row는 Series 객체
for idx,row in weather_df.iterrows():
        #row['datas']는 list객체
    for idx,data in enumerate(row['datas'],1):
        #data는 dict객체
        for key,value in data.items():
            #mode -> mode1 로 변경
            new_key = key + str(idx)
            #컬럼을 생성
            weather_df[new_key] = value

print(weather_df.shape)            
print(weather_df.columns)

(41, 68)
Index(['province', 'city', 'datas', 'mode1', 'tmef1', 'wf1', 'tmn1', 'tmx1',
       'mode2', 'tmef2', 'wf2', 'tmn2', 'tmx2', 'mode3', 'tmef3', 'wf3',
       'tmn3', 'tmx3', 'mode4', 'tmef4', 'wf4', 'tmn4', 'tmx4', 'mode5',
       'tmef5', 'wf5', 'tmn5', 'tmx5', 'mode6', 'tmef6', 'wf6', 'tmn6', 'tmx6',
       'mode7', 'tmef7', 'wf7', 'tmn7', 'tmx7', 'mode8', 'tmef8', 'wf8',
       'tmn8', 'tmx8', 'mode9', 'tmef9', 'wf9', 'tmn9', 'tmx9', 'mode10',
       'tmef10', 'wf10', 'tmn10', 'tmx10', 'mode11', 'tmef11', 'wf11', 'tmn11',
       'tmx11', 'mode12', 'tmef12', 'wf12', 'tmn12', 'tmx12', 'mode13',
       'tmef13', 'wf13', 'tmn13', 'tmx13'],
      dtype='object')


In [117]:
weather_df.head(2)

,province,city,datas,mode1,tmef1,wf1,tmn1,tmx1,mode2,tmef2,...,mode12,tmef12,wf12,tmn12,tmx12,mode13,tmef13,wf13,tmn13,tmx13
0,서울ㆍ인천ㆍ경기도,서울,"[{'mode': 'A02', 'tmef': '2020-10-24 00:00', '...",A02,2020-10-24 00:00,구름많음,11,19,A02,2020-10-24 12:00,...,A01,2020-10-30 00:00,맑음,14,21,A01,2020-10-31 00:00,맑음,16,22
1,서울ㆍ인천ㆍ경기도,인천,"[{'mode': 'A02', 'tmef': '2020-10-24 00:00', '...",A02,2020-10-24 00:00,구름많음,11,19,A02,2020-10-24 12:00,...,A01,2020-10-30 00:00,맑음,14,21,A01,2020-10-31 00:00,맑음,16,22


In [118]:
#datas 컬럼을 삭제
#drop함수는 행 이나 열을 삭제하는 함수
#행을 삭제하면 axis = 0, 열을 삭제하면 axis = 1
#inplace=False 가 default이고, 실제 삭제가 되지 않고, 임시적으로 삭제된 결과를 출력
#inplace=True이면 실제가 삭제가 되고, 삭제된 결과를 출력하지 않는다.
weather_df.drop('datas', axis=1)
#print(weather_df.shape)

,province,city,mode1,tmef1,wf1,tmn1,tmx1,mode2,tmef2,wf2,...,mode12,tmef12,wf12,tmn12,tmx12,mode13,tmef13,wf13,tmn13,tmx13
0,서울ㆍ인천ㆍ경기도,서울,A02,2020-10-24 00:00,구름많음,11,19,A02,2020-10-24 12:00,구름많음,...,A01,2020-10-30 00:00,맑음,14,21,A01,2020-10-31 00:00,맑음,16,22
1,서울ㆍ인천ㆍ경기도,인천,A02,2020-10-24 00:00,구름많음,11,19,A02,2020-10-24 12:00,구름많음,...,A01,2020-10-30 00:00,맑음,14,21,A01,2020-10-31 00:00,맑음,16,22
2,서울ㆍ인천ㆍ경기도,수원,A02,2020-10-24 00:00,구름많음,11,19,A02,2020-10-24 12:00,구름많음,...,A01,2020-10-30 00:00,맑음,14,21,A01,2020-10-31 00:00,맑음,16,22
3,서울ㆍ인천ㆍ경기도,파주,A02,2020-10-24 00:00,구름많음,11,19,A02,2020-10-24 12:00,구름많음,...,A01,2020-10-30 00:00,맑음,14,21,A01,2020-10-31 00:00,맑음,16,22
4,서울ㆍ인천ㆍ경기도,이천,A02,2020-10-24 00:00,구름많음,11,19,A02,2020-10-24 12:00,구름많음,...,A01,2020-10-30 00:00,맑음,14,21,A01,2020-10-31 00:00,맑음,16,22
5,서울ㆍ인천ㆍ경기도,평택,A02,2020-10-24 00:00,구름많음,11,19,A02,2020-10-24 12:00,구름많음,...,A01,2020-10-30 00:00,맑음,14,21,A01,2020-10-31 00:00,맑음,16,22
6,강원도영서,춘천,A02,2020-10-24 00:00,구름많음,11,19,A02,2020-10-24 12:00,구름많음,...,A01,2020-10-30 00:00,맑음,14,21,A01,2020-10-31 00:00,맑음,16,22
7,강원도영서,원주,A02,2020-10-24 00:00,구름많음,11,19,A02,2020-10-24 12:00,구름많음,...,A01,2020-10-30 00:00,맑음,14,21,A01,2020-10-31 00:00,맑음,16,22
8,강원도영동,강릉,A02,2020-10-24 00:00,구름많음,11,19,A02,2020-10-24 12:00,구름많음,...,A01,2020-10-30 00:00,맑음,14,21,A01,2020-10-31 00:00,맑음,16,22
9,대전ㆍ세종ㆍ충청남도,대전,A02,2020-10-24 00:00,구름많음,11,19,A02,2020-10-24 12:00,구름많음,...,A01,2020-10-30 00:00,맑음,14,21,A01,2020-10-31 00:00,맑음,16,22


In [119]:
weather_df.drop('datas', axis=1, inplace=True)
weather_df.head(1)

,province,city,mode1,tmef1,wf1,tmn1,tmx1,mode2,tmef2,wf2,...,mode12,tmef12,wf12,tmn12,tmx12,mode13,tmef13,wf13,tmn13,tmx13
0,서울ㆍ인천ㆍ경기도,서울,A02,2020-10-24 00:00,구름많음,11,19,A02,2020-10-24 12:00,구름많음,...,A01,2020-10-30 00:00,맑음,14,21,A01,2020-10-31 00:00,맑음,16,22


In [120]:
#province 컬럼을 index로 변경하기
weather_df = weather_df.set_index('province')
weather_df.head(2)

,city,mode1,tmef1,wf1,tmn1,tmx1,mode2,tmef2,wf2,tmn2,...,mode12,tmef12,wf12,tmn12,tmx12,mode13,tmef13,wf13,tmn13,tmx13
province,,,,,,,,,,,,,,,,,,,,,
서울ㆍ인천ㆍ경기도,서울,A02,2020-10-24 00:00,구름많음,11,19,A02,2020-10-24 12:00,구름많음,11,...,A01,2020-10-30 00:00,맑음,14,21,A01,2020-10-31 00:00,맑음,16,22
서울ㆍ인천ㆍ경기도,인천,A02,2020-10-24 00:00,구름많음,11,19,A02,2020-10-24 12:00,구름많음,11,...,A01,2020-10-30 00:00,맑음,14,21,A01,2020-10-31 00:00,맑음,16,22


In [122]:
weather_df.index

Index(['서울ㆍ인천ㆍ경기도', '서울ㆍ인천ㆍ경기도', '서울ㆍ인천ㆍ경기도', '서울ㆍ인천ㆍ경기도', '서울ㆍ인천ㆍ경기도',
       '서울ㆍ인천ㆍ경기도', '강원도영서', '강원도영서', '강원도영동', '대전ㆍ세종ㆍ충청남도', '대전ㆍ세종ㆍ충청남도',
       '대전ㆍ세종ㆍ충청남도', '충청북도', '충청북도', '충청북도', '광주ㆍ전라남도', '광주ㆍ전라남도', '광주ㆍ전라남도',
       '광주ㆍ전라남도', '광주ㆍ전라남도', '광주ㆍ전라남도', '전라북도', '전라북도', '전라북도', '전라북도', '전라북도',
       '전라북도', '부산ㆍ울산ㆍ경상남도', '부산ㆍ울산ㆍ경상남도', '부산ㆍ울산ㆍ경상남도', '부산ㆍ울산ㆍ경상남도',
       '부산ㆍ울산ㆍ경상남도', '부산ㆍ울산ㆍ경상남도', '대구ㆍ경상북도', '대구ㆍ경상북도', '대구ㆍ경상북도', '대구ㆍ경상북도',
       '대구ㆍ경상북도', '대구ㆍ경상북도', '제주도', '제주도'],
      dtype='object', name='province')

In [124]:
weather_df.loc['서울ㆍ인천ㆍ경기도']

,city,mode1,tmef1,wf1,tmn1,tmx1,mode2,tmef2,wf2,tmn2,...,mode12,tmef12,wf12,tmn12,tmx12,mode13,tmef13,wf13,tmn13,tmx13
province,,,,,,,,,,,,,,,,,,,,,
서울ㆍ인천ㆍ경기도,서울,A02,2020-10-24 00:00,구름많음,11,19,A02,2020-10-24 12:00,구름많음,11,...,A01,2020-10-30 00:00,맑음,14,21,A01,2020-10-31 00:00,맑음,16,22
서울ㆍ인천ㆍ경기도,인천,A02,2020-10-24 00:00,구름많음,11,19,A02,2020-10-24 12:00,구름많음,11,...,A01,2020-10-30 00:00,맑음,14,21,A01,2020-10-31 00:00,맑음,16,22
서울ㆍ인천ㆍ경기도,수원,A02,2020-10-24 00:00,구름많음,11,19,A02,2020-10-24 12:00,구름많음,11,...,A01,2020-10-30 00:00,맑음,14,21,A01,2020-10-31 00:00,맑음,16,22
서울ㆍ인천ㆍ경기도,파주,A02,2020-10-24 00:00,구름많음,11,19,A02,2020-10-24 12:00,구름많음,11,...,A01,2020-10-30 00:00,맑음,14,21,A01,2020-10-31 00:00,맑음,16,22
서울ㆍ인천ㆍ경기도,이천,A02,2020-10-24 00:00,구름많음,11,19,A02,2020-10-24 12:00,구름많음,11,...,A01,2020-10-30 00:00,맑음,14,21,A01,2020-10-31 00:00,맑음,16,22
서울ㆍ인천ㆍ경기도,평택,A02,2020-10-24 00:00,구름많음,11,19,A02,2020-10-24 12:00,구름많음,11,...,A01,2020-10-30 00:00,맑음,14,21,A01,2020-10-31 00:00,맑음,16,22


In [126]:
weather_df.T

province,서울ㆍ인천ㆍ경기도,서울ㆍ인천ㆍ경기도,서울ㆍ인천ㆍ경기도,서울ㆍ인천ㆍ경기도,서울ㆍ인천ㆍ경기도,서울ㆍ인천ㆍ경기도,강원도영서,강원도영서,강원도영동,대전ㆍ세종ㆍ충청남도,...,부산ㆍ울산ㆍ경상남도,부산ㆍ울산ㆍ경상남도,대구ㆍ경상북도,대구ㆍ경상북도,대구ㆍ경상북도,대구ㆍ경상북도,대구ㆍ경상북도,대구ㆍ경상북도,제주도,제주도
city,서울,인천,수원,파주,이천,평택,춘천,원주,강릉,대전,...,거창,통영,대구,안동,포항,경주,울진,울릉도,제주,서귀포
mode1,A02,A02,A02,A02,A02,A02,A02,A02,A02,A02,...,A02,A02,A02,A02,A02,A02,A02,A02,A02,A02
tmef1,2020-10-24 00:00,2020-10-24 00:00,2020-10-24 00:00,2020-10-24 00:00,2020-10-24 00:00,2020-10-24 00:00,2020-10-24 00:00,2020-10-24 00:00,2020-10-24 00:00,2020-10-24 00:00,...,2020-10-24 00:00,2020-10-24 00:00,2020-10-24 00:00,2020-10-24 00:00,2020-10-24 00:00,2020-10-24 00:00,2020-10-24 00:00,2020-10-24 00:00,2020-10-24 00:00,2020-10-24 00:00
wf1,구름많음,구름많음,구름많음,구름많음,구름많음,구름많음,구름많음,구름많음,구름많음,구름많음,...,구름많음,구름많음,구름많음,구름많음,구름많음,구름많음,구름많음,구름많음,구름많음,구름많음
tmn1,11,11,11,11,11,11,11,11,11,11,...,11,11,11,11,11,11,11,11,11,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
mode13,A01,A01,A01,A01,A01,A01,A01,A01,A01,A01,...,A01,A01,A01,A01,A01,A01,A01,A01,A01,A01
tmef13,2020-10-31 00:00,2020-10-31 00:00,2020-10-31 00:00,2020-10-31 00:00,2020-10-31 00:00,2020-10-31 00:00,2020-10-31 00:00,2020-10-31 00:00,2020-10-31 00:00,2020-10-31 00:00,...,2020-10-31 00:00,2020-10-31 00:00,2020-10-31 00:00,2020-10-31 00:00,2020-10-31 00:00,2020-10-31 00:00,2020-10-31 00:00,2020-10-31 00:00,2020-10-31 00:00,2020-10-31 00:00
wf13,맑음,맑음,맑음,맑음,맑음,맑음,맑음,맑음,맑음,맑음,...,맑음,맑음,맑음,맑음,맑음,맑음,맑음,맑음,맑음,맑음
tmn13,16,16,16,16,16,16,16,16,16,16,...,16,16,16,16,16,16,16,16,16,16


In [128]:
weather_df.transpose()

province,서울ㆍ인천ㆍ경기도,서울ㆍ인천ㆍ경기도,서울ㆍ인천ㆍ경기도,서울ㆍ인천ㆍ경기도,서울ㆍ인천ㆍ경기도,서울ㆍ인천ㆍ경기도,강원도영서,강원도영서,강원도영동,대전ㆍ세종ㆍ충청남도,...,부산ㆍ울산ㆍ경상남도,부산ㆍ울산ㆍ경상남도,대구ㆍ경상북도,대구ㆍ경상북도,대구ㆍ경상북도,대구ㆍ경상북도,대구ㆍ경상북도,대구ㆍ경상북도,제주도,제주도
city,서울,인천,수원,파주,이천,평택,춘천,원주,강릉,대전,...,거창,통영,대구,안동,포항,경주,울진,울릉도,제주,서귀포
mode1,A02,A02,A02,A02,A02,A02,A02,A02,A02,A02,...,A02,A02,A02,A02,A02,A02,A02,A02,A02,A02
tmef1,2020-10-24 00:00,2020-10-24 00:00,2020-10-24 00:00,2020-10-24 00:00,2020-10-24 00:00,2020-10-24 00:00,2020-10-24 00:00,2020-10-24 00:00,2020-10-24 00:00,2020-10-24 00:00,...,2020-10-24 00:00,2020-10-24 00:00,2020-10-24 00:00,2020-10-24 00:00,2020-10-24 00:00,2020-10-24 00:00,2020-10-24 00:00,2020-10-24 00:00,2020-10-24 00:00,2020-10-24 00:00
wf1,구름많음,구름많음,구름많음,구름많음,구름많음,구름많음,구름많음,구름많음,구름많음,구름많음,...,구름많음,구름많음,구름많음,구름많음,구름많음,구름많음,구름많음,구름많음,구름많음,구름많음
tmn1,11,11,11,11,11,11,11,11,11,11,...,11,11,11,11,11,11,11,11,11,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
mode13,A01,A01,A01,A01,A01,A01,A01,A01,A01,A01,...,A01,A01,A01,A01,A01,A01,A01,A01,A01,A01
tmef13,2020-10-31 00:00,2020-10-31 00:00,2020-10-31 00:00,2020-10-31 00:00,2020-10-31 00:00,2020-10-31 00:00,2020-10-31 00:00,2020-10-31 00:00,2020-10-31 00:00,2020-10-31 00:00,...,2020-10-31 00:00,2020-10-31 00:00,2020-10-31 00:00,2020-10-31 00:00,2020-10-31 00:00,2020-10-31 00:00,2020-10-31 00:00,2020-10-31 00:00,2020-10-31 00:00,2020-10-31 00:00
wf13,맑음,맑음,맑음,맑음,맑음,맑음,맑음,맑음,맑음,맑음,...,맑음,맑음,맑음,맑음,맑음,맑음,맑음,맑음,맑음,맑음
tmn13,16,16,16,16,16,16,16,16,16,16,...,16,16,16,16,16,16,16,16,16,16


In [131]:
weather = weather_df.transpose()
weather.head()

province,서울ㆍ인천ㆍ경기도,서울ㆍ인천ㆍ경기도,서울ㆍ인천ㆍ경기도,서울ㆍ인천ㆍ경기도,서울ㆍ인천ㆍ경기도,서울ㆍ인천ㆍ경기도,강원도영서,강원도영서,강원도영동,대전ㆍ세종ㆍ충청남도,...,부산ㆍ울산ㆍ경상남도,부산ㆍ울산ㆍ경상남도,대구ㆍ경상북도,대구ㆍ경상북도,대구ㆍ경상북도,대구ㆍ경상북도,대구ㆍ경상북도,대구ㆍ경상북도,제주도,제주도
city,서울,인천,수원,파주,이천,평택,춘천,원주,강릉,대전,...,거창,통영,대구,안동,포항,경주,울진,울릉도,제주,서귀포
mode1,A02,A02,A02,A02,A02,A02,A02,A02,A02,A02,...,A02,A02,A02,A02,A02,A02,A02,A02,A02,A02
tmef1,2020-10-24 00:00,2020-10-24 00:00,2020-10-24 00:00,2020-10-24 00:00,2020-10-24 00:00,2020-10-24 00:00,2020-10-24 00:00,2020-10-24 00:00,2020-10-24 00:00,2020-10-24 00:00,...,2020-10-24 00:00,2020-10-24 00:00,2020-10-24 00:00,2020-10-24 00:00,2020-10-24 00:00,2020-10-24 00:00,2020-10-24 00:00,2020-10-24 00:00,2020-10-24 00:00,2020-10-24 00:00
wf1,구름많음,구름많음,구름많음,구름많음,구름많음,구름많음,구름많음,구름많음,구름많음,구름많음,...,구름많음,구름많음,구름많음,구름많음,구름많음,구름많음,구름많음,구름많음,구름많음,구름많음
tmn1,11,11,11,11,11,11,11,11,11,11,...,11,11,11,11,11,11,11,11,11,11


In [132]:
# 춘천의 날씨 날씨는?
weather['강원도영서']

province,강원도영서,강원도영서
city,춘천,원주
mode1,A02,A02
tmef1,2020-10-24 00:00,2020-10-24 00:00
wf1,구름많음,구름많음
tmn1,11,11
...,...,...
mode13,A01,A01
tmef13,2020-10-31 00:00,2020-10-31 00:00
wf13,맑음,맑음
tmn13,16,16
